In [1]:
import os
import sys
import datetime
import time
import json
import re
from collections import defaultdict
import glob

def read_csv_to_dict( input_table ):
    list_return = []

    list_header = []

    with open(input_table, 'r') as f:
        run_flag = 0

        for line in f:
            run_flag = run_flag + 1

            line = line.strip()

            if run_flag == 1:
                list_header = line.split(',')

            else:
                list_row_data = line.split(',')
                if len(list_row_data) == 0:
                    continue
                    
                dict_row_data = {}
                for i in range(len(list_header)):
                    dict_row_data[ list_header[i] ] = list_row_data[i]

                list_return.append( dict_row_data )

    return list_return

def write_dict_to_csv(file_open_option, list_datas, list_header, output_file_path ):
    
    # https://stackoverflow.com/questions/12517451/automatically-creating-directories-with-file-output
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
    
    #if os.path.isfile(output_file_path):
    #    file_open_option = 'a'
    #else:
    #    file_open_option = 'w'
    
    with open(output_file_path, file_open_option) as f:
        if file_open_option == 'w':
            f.write( ','.join(list_header)  + '\n')

        for this_data in list_datas:
            run_flag = 0
            output_string = ''

            for this_key in list_header:
                run_flag = run_flag + 1
                if run_flag == 1:
                    output_string = str(this_data[this_key])
                else:
                    output_string = output_string + ',' + str(this_data[this_key])

            f.write(output_string + '\n')

def write_list_to_csv( list_datas, list_header, output_file_path ):
    
    with open(output_file_path, 'w') as f:
        f.write( ','.join(list_header)  + '\n')

        for this_data in list_datas:
            f.write( ','.join(this_data)  + '\n')

def read_file_to_list( input_table ):
    list_return = []

    with open(input_table, 'r') as f:
        run_flag = 0

        for line in f:
            run_flag = run_flag + 1

            line = line.strip()
            list_return.append( line )

    return list_return
            
def get_list_file_path( path_representation ):
    #list = glob.glob('/home/raohongfu/*.php')
    list_return = glob.glob( path_representation  )
    #for this_path in list_return:
    #    print(this_path)
    return list_return
            
def get_list_symbol_from_outputs():
    file_path = './outputs_manual_train/tw/*'
    list_symbol_path = get_list_file_path( file_path  )

    list_return = []
    for this_path in list_symbol_path:
        list_temp = this_path.split('/')
        list_return.append( list_temp[-1] )

    return sorted(list_return)

def get_list_valid_date():
    #file_path = './outputs_manual_train/stats/valid_date.txt'
    file_path = './outputs_manual_train/run.log'
    list_lines = read_file_to_list(file_path)
    
    list_valid_date = []
    for line in list_lines:
        list_valid_date.append(line.split(' ')[0])
        
    # return last 3 date
    return list_valid_date[-8:]

def get_list_date_group_string():
    # return list of 20180808_20180809_20180810 , 20180809_20180810_20180811 , ....
    list_date = get_list_valid_date()
    day_count = 3
    
    # get group of continuous dates by day_count
    list_group_of_date = []
    for i in range(len(list_date)):
        start_index = i
        end_index = start_index + day_count        
        #print(list_date[start_index:end_index])
        # we can break for all days has its group
        if end_index > len(list_date):
            break            
        list_group_of_date.append(list_date[start_index:end_index])
        
    list_date_group_string = []
    for this_date_group in list_group_of_date:
        list_date_group_string.append( '_'.join(this_date_group) )
        
    # last one no need return due to no real plus_1_day and plus_2_day data to be compared
    return list_date_group_string
            
def prepare_data():
    # load dataset
    input_file_path = './outputs_manual_train/stats/profile_qty_price_multiple_day/profile_qty_price_multiple_day_for_predict.csv'
    original_dataset = read_csv(input_file_path, header=0, index_col=0, \
                       dtype={'symbol': str, \
                              'date_group': str, \
                              'price_1': float, \
                              'qty_ratio_1': float, \
                              'price_2': float, \
                              'qty_ratio_2': float, \
                              'price_3': float, \
                              'qty_ratio_3': float, \
                              'qty_total': float, \
                              'price_item_count': float, \
                              'plus_1_day_low_price': float, \
                              'plus_1_day_high_price': float, \
                              'plus_2_day_low_price': float, \
                              'plus_2_day_high_price': float
                             })
    
    #display(original_dataset)
    
    # used for write csv along with predicted prices
    nparray_column_to_keep_for_merge = original_dataset.values[ : , : -4]
    
    dataset = original_dataset
    #https://stackoverflow.com/questions/13411544/delete-column-from-pandas-dataframe-using-del-df-column-name
    dataset = dataset.drop('symbol', 1)
    dataset = dataset.drop('date_group', 1)
    #display(dataset)
    dataset_predict_plus_1_day_low_price = dataset
    dataset_predict_plus_1_day_low_price = dataset_predict_plus_1_day_low_price.drop('plus_1_day_high_price', 1)
    dataset_predict_plus_1_day_low_price = dataset_predict_plus_1_day_low_price.drop('plus_2_day_low_price', 1)
    dataset_predict_plus_1_day_low_price = dataset_predict_plus_1_day_low_price.drop('plus_2_day_high_price', 1)
    
    dataset_predict_plus_1_day_high_price = dataset
    dataset_predict_plus_1_day_high_price = dataset_predict_plus_1_day_high_price.drop('plus_1_day_low_price', 1)
    dataset_predict_plus_1_day_high_price = dataset_predict_plus_1_day_high_price.drop('plus_2_day_low_price', 1)
    dataset_predict_plus_1_day_high_price = dataset_predict_plus_1_day_high_price.drop('plus_2_day_high_price', 1)
    
    dataset_predict_plus_2_day_low_price = dataset
    dataset_predict_plus_2_day_low_price = dataset_predict_plus_2_day_low_price.drop('plus_1_day_low_price', 1)
    dataset_predict_plus_2_day_low_price = dataset_predict_plus_2_day_low_price.drop('plus_1_day_high_price', 1)
    dataset_predict_plus_2_day_low_price = dataset_predict_plus_2_day_low_price.drop('plus_2_day_high_price', 1)
    
    dataset_predict_plus_2_day_high_price = dataset
    dataset_predict_plus_2_day_high_price = dataset_predict_plus_2_day_high_price.drop('plus_1_day_low_price', 1)
    dataset_predict_plus_2_day_high_price = dataset_predict_plus_2_day_high_price.drop('plus_1_day_high_price', 1)
    dataset_predict_plus_2_day_high_price = dataset_predict_plus_2_day_high_price.drop('plus_2_day_low_price', 1)
    
    nparray_predict_plus_1_day_low_price = dataset_predict_plus_1_day_low_price.values
    nparray_predict_plus_1_day_high_price = dataset_predict_plus_1_day_high_price.values
    nparray_predict_plus_2_day_low_price = dataset_predict_plus_2_day_low_price.values
    nparray_predict_plus_2_day_high_price = dataset_predict_plus_2_day_high_price.values

    return nparray_column_to_keep_for_merge , nparray_predict_plus_1_day_low_price , nparray_predict_plus_1_day_high_price, nparray_predict_plus_2_day_low_price, nparray_predict_plus_2_day_high_price

def predict_with_model(nparray_input_data, scaler, model):

    # plot sequential_test_data_of_stocks
    #print(nparray_input_data.shape)
    nparray_input_data_scaled = scaler.transform(nparray_input_data)

    # define columns for test_X, test_Y
    n_cols_outputs = 1
    n_cols_inputs = nparray_input_data_scaled.shape[1] - n_cols_outputs
    
    # split to test_X , test_Y
    test_X, test_Y = nparray_input_data_scaled[:, :n_cols_inputs], nparray_input_data_scaled[:, -n_cols_outputs]

    # reshape input to be 3D [samples, timesteps, features]  <----- LSTM , if MLP no need reshape
    test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
    
    # make a prediction
    y_predict = model.predict(test_X)

    # reshape back original shape
    test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
    
    # concatenate to original data shape
    nparray_predicted_data_scaled = numpy.concatenate((test_X[:, :], y_predict), axis=1)
    
    # scale back to real value
    nparray_predicted_data_real = scaler.inverse_transform(nparray_predicted_data_scaled)
    
    return nparray_predicted_data_real


def predict_future_prices():
    
    print('\n\n')
    print('.....................................................................................................................')
    print('.                Predict_future_sell_price Run                                                                      .')
    print('.....................................................................................................................')
    
    print('\nprepare data ..')
    nparray_column_to_keep_for_merge , nparray_predict_plus_1_day_low_price , nparray_predict_plus_1_day_high_price, nparray_predict_plus_2_day_low_price, nparray_predict_plus_2_day_high_price = prepare_data()
    print(nparray_column_to_keep_for_merge.shape)
    
    # --- nparray_predict_plus_1_day_low_price ---------------------------------------------
    # load scaler
    print('\nload scaler (finalized_scaler_plus_1_day_low_price.sav) ..')
    scaler_filename = './finalized_scaler_plus_1_day_low_price.sav'
    scaler = pickle.load(open(scaler_filename, 'rb'))
    
    # load model
    print('\nload model (finalized_model_plus_1_day_low_price.sav) ..')
    model_filename = './finalized_model_plus_1_day_low_price.sav'
    model = load_model(model_filename)
    
    # predict future sell price
    print('\npredict plus_1_day_low_price ............................\n')
    nparray_predict_plus_1_day_low_price_real = predict_with_model(nparray_predict_plus_1_day_low_price, scaler, model)
    nparray_predict_plus_1_day_low_price_real = nparray_predict_plus_1_day_low_price_real[:,-1]
    nparray_predict_plus_1_day_low_price_real = nparray_predict_plus_1_day_low_price_real.reshape((len(nparray_predict_plus_1_day_low_price_real), 1))
    print(nparray_predict_plus_1_day_low_price_real.shape)
      
    # create output data nparray
    print('\nmerge plus_1_day_low_price ............................\n')
    nparray_output_data = numpy.concatenate((nparray_column_to_keep_for_merge[:, :], nparray_predict_plus_1_day_low_price_real), axis=1)
    print(nparray_output_data.shape)
    
    # --- nparray_predict_plus_1_day_high_price ---------------------------------------------
    # load scaler
    print('\nload scaler (finalized_scaler_plus_1_day_high_price.sav) ..')
    scaler_filename = './finalized_scaler_plus_1_day_high_price.sav'
    scaler = pickle.load(open(scaler_filename, 'rb'))
    
    # load model
    print('\nload model (finalized_model_plus_1_day_high_price.sav) ..')
    model_filename = './finalized_model_plus_1_day_high_price.sav'
    model = load_model(model_filename)
    
    # predict future sell price
    print('\npredict plus_1_day_high_price ............................\n')
    nparray_predict_plus_1_day_high_price_real = predict_with_model(nparray_predict_plus_1_day_high_price, scaler, model)
    nparray_predict_plus_1_day_high_price_real = nparray_predict_plus_1_day_high_price_real[:,-1]
    nparray_predict_plus_1_day_high_price_real = nparray_predict_plus_1_day_high_price_real.reshape((len(nparray_predict_plus_1_day_high_price_real), 1))
    print(nparray_predict_plus_1_day_high_price_real.shape)
      
    # create output data nparray
    print('\nmerge plus_1_day_high_price ............................\n')
    nparray_output_data = numpy.concatenate((nparray_output_data[:, :], nparray_predict_plus_1_day_high_price_real), axis=1)
    print(nparray_output_data.shape)
    
    # --- nparray_predict_plus_2_day_low_price ---------------------------------------------
    # load scaler
    print('\nload scaler (finalized_scaler_plus_2_day_low_price.sav) ..')
    scaler_filename = './finalized_scaler_plus_2_day_low_price.sav'
    scaler = pickle.load(open(scaler_filename, 'rb'))
    
    # load model
    print('\nload model (finalized_model_plus_2_day_low_price.sav) ..')
    model_filename = './finalized_model_plus_2_day_low_price.sav'
    model = load_model(model_filename)
    
    # predict future sell price
    print('\npredict plus_2_day_low_price ............................\n')
    nparray_predict_plus_2_day_low_price_real = predict_with_model(nparray_predict_plus_2_day_low_price, scaler, model)
    nparray_predict_plus_2_day_low_price_real = nparray_predict_plus_2_day_low_price_real[:,-1]
    nparray_predict_plus_2_day_low_price_real = nparray_predict_plus_2_day_low_price_real.reshape((len(nparray_predict_plus_2_day_low_price_real), 1))
    print(nparray_predict_plus_2_day_low_price_real.shape)
      
    # create output data nparray
    print('\nmerge plus_2_day_low_price ............................\n')
    nparray_output_data = numpy.concatenate((nparray_output_data[:, :], nparray_predict_plus_2_day_low_price_real), axis=1)
    print(nparray_output_data.shape)
    
    # --- nparray_predict_plus_2_day_high_price ---------------------------------------------
    # load scaler
    print('\nload scaler (finalized_scaler_plus_2_day_high_price.sav) ..')
    scaler_filename = './finalized_scaler_plus_2_day_high_price.sav'
    scaler = pickle.load(open(scaler_filename, 'rb'))
    
    # load model
    print('\nload model (finalized_model_plus_2_day_high_price.sav) ..')
    model_filename = './finalized_model_plus_2_day_high_price.sav'
    model = load_model(model_filename)
    
    # predict future sell price
    print('\npredict plus_2_day_high_price ............................\n')
    nparray_predict_plus_2_day_high_price_real = predict_with_model(nparray_predict_plus_2_day_high_price, scaler, model)
    nparray_predict_plus_2_day_high_price_real = nparray_predict_plus_2_day_high_price_real[:,-1]
    nparray_predict_plus_2_day_high_price_real = nparray_predict_plus_2_day_high_price_real.reshape((len(nparray_predict_plus_2_day_high_price_real), 1))
    print(nparray_predict_plus_2_day_high_price_real.shape)
      
    # create output data nparray
    print('\nmerge plus_2_day_high_price ............................\n')
    nparray_output_data = numpy.concatenate((nparray_output_data[:, :], nparray_predict_plus_2_day_high_price_real), axis=1)
    print(nparray_output_data.shape)
    
    
    # write to file
    list_header = ['symbol', 'date_group', 'price_1', 'qty_ratio_1', 'price_2', 'qty_ratio_2', 'price_3', 'qty_ratio_3', 'qty_total', 'price_item_count', 'plus_1_day_low_price', 'plus_1_day_high_price', 'plus_2_day_low_price', 'plus_2_day_high_price']
    list_output_datas = nparray_output_data.tolist()
    for i in range(len(list_output_datas)):
        for j in range(2, len(list_output_datas[i])):
            list_output_datas[i][j] = '{:.2f}'.format(list_output_datas[i][j])
    
    today = datetime.datetime.today().strftime('%Y_%m_%d')
    output_file_path = './predict_future_prices/' + today + '.csv'
    write_list_to_csv( list_output_datas, list_header, output_file_path )  
        
    return

if __name__ == '__main__':
    
    # get list of symbols
    list_symbol = get_list_symbol_from_outputs()
    
    # read_daily_data
    file_profile_qty_price_multiple_day_for_machine_learning = './outputs_manual_train/stats/profile_qty_price_multiple_day/profile_qty_price_multiple_day_for_machine_learning.csv'
    list_real_daily_data = read_csv_to_dict( file_profile_qty_price_multiple_day_for_machine_learning )
    
    # predict_daily_data
    today = datetime.datetime.today().strftime('%Y_%m_%d')
    file_predict_daily_data = './predict_future_prices/future_prices_' + today + '.csv'
    list_predict_daily_data = read_csv_to_dict( file_predict_daily_data )
    
    # list of 20180808_20180809_20180810 , 20180809_20180810_20180811 , ....
    list_date_group_string = get_list_date_group_string()
    
    list_output_datas = []
    for this_symbol in list_symbol:
        print('\n\nprocess ' + this_symbol + ' for ', end='')
        
        #https://stackoverflow.com/questions/17215400/python-format-string-unused-named-arguments
        #'{bond}, {james} {bond}'.format_map(defaultdict(str, bond='bond'))
        dict_output = {}
        dict_output['symbol'] = this_symbol
        dict_output['plus_1_day_low_price_offset_past_days'] = ''
        dict_output['plus_1_day_high_price_offset_past_days'] = ''
        dict_output['plus_2_day_low_price_offset_past_days'] = ''
        dict_output['plus_2_day_high_price_offset_past_days'] = ''
        dict_output['plus_1_day_low_price_predict'] = ''
        dict_output['plus_1_day_high_price_predict'] = ''
        dict_output['plus_2_day_low_price_predict'] = ''
        dict_output['plus_2_day_high_price_predict'] = ''
        
        # loop list_date_group_string to calculate average abs_offset 
        list_plus_1_day_low_price_offset_past_days = []
        list_plus_1_day_high_price_offset_past_days = []
        list_plus_2_day_low_price_offset_past_days = []
        list_plus_2_day_high_price_offset_past_days = []
        
        # check if there is today's predict data, it is meaningless if can't not predict future data
        this_predict_date_group_string = list_date_group_string[-1]
        print(this_predict_date_group_string)
        list_temp = [x for x in list_predict_daily_data if x['symbol'] == this_symbol and x['date_group'] == this_predict_date_group_string]
        if len(list_temp) == 0:
            print('\tno predictable data.', end = '\n')
            continue
        
        list_date_group_string_real_data_range = list_date_group_string[:-2]
        for i in range( len(list_date_group_string_real_data_range) ):
            
            this_date_group_string = list_date_group_string[i]
            print('\t' + this_date_group_string, end='')
            
            this_predict_data_of_this_date_group = next(x for x in list_predict_daily_data if x['symbol'] == this_symbol and x['date_group'] == this_predict_date_group_string)
        
            
            list_temp = [x for x in list_real_daily_data if x['symbol'] == this_symbol and x['date_group'] == this_date_group_string]
            if len(list_temp) == 1:
                this_real_data_of_this_date_group = list_temp[0]
            else:
                print(' no real data.', end = '\n')
                continue
                
            list_temp = [x for x in list_predict_daily_data if x['symbol'] == this_symbol and x['date_group'] == this_date_group_string]
            if len(list_temp) == 1:
                this_predict_data_of_this_date_group = list_temp[0]
            else:
                print(' no predict data.', end = '\n')
                continue
                
            # if run to this line, means there is data to compare between read-data and predict-data
            print(' has data.', end = '\n')
            
            this_real_plus_1_day_low_price = this_real_data_of_this_date_group['plus_1_day_low_price']
            this_real_plus_1_day_high_price = this_real_data_of_this_date_group['plus_1_day_high_price']
            this_real_plus_2_day_low_price = this_real_data_of_this_date_group['plus_2_day_low_price']
            this_real_plus_2_day_high_price = this_real_data_of_this_date_group['plus_2_day_high_price']
        
            this_predict_plus_1_day_low_price = this_predict_data_of_this_date_group['plus_1_day_low_price']
            this_predict_plus_1_day_high_price = this_predict_data_of_this_date_group['plus_1_day_high_price']
            this_predict_plus_2_day_low_price = this_predict_data_of_this_date_group['plus_2_day_low_price']
            this_predict_plus_2_day_high_price = this_predict_data_of_this_date_group['plus_2_day_high_price']
            
            this_plus_1_day_low_price_abs_offset = abs(this_predict_plus_1_day_low_price - this_real_plus_1_day_low_price) / this_real_plus_1_day_low_price
            this_plus_1_day_high_price_abs_offset = abs(this_predict_plus_1_day_high_price - this_real_plus_1_day_high_price) / this_real_plus_1_day_high_price
            this_plus_2_day_low_price_abs_offset = abs(this_predict_plus_2_day_low_price - this_real_plus_2_day_low_price) / this_real_plus_2_day_low_price
            this_plus_2_day_high_price_abs_offset = abs(this_predict_plus_2_day_high_price - this_real_plus_2_day_high_price) / this_real_plus_2_day_high_price
            
            list_plus_1_day_low_price_offset_past_days.append(this_plus_1_day_low_price_abs_offset) 
            list_plus_1_day_high_price_offset_past_days.append(this_plus_1_day_high_price_abs_offset) 
            list_plus_2_day_low_price_offset_past_days.append(this_plus_2_day_low_price_abs_offset) 
            list_plus_2_day_high_price_offset_past_days.append(this_plus_2_day_high_price_abs_offset) 
        
        print('list_plus_2_day_low_price_offset_past_days: ' + str(len(list_plus_2_day_low_price_offset_past_days)))
        
        # have average offset example as 1.05 %
        average_plus_1_day_low_price_offset_past_days = -999
        average_plus_1_day_high_price_offset_past_days = -999
        average_plus_2_day_low_price_offset_past_days = -999
        average_plus_2_day_high_price_offset_past_days = -999
        if len(list_plus_1_day_low_price_offset_past_days) > 0:
            average_plus_1_day_low_price_offset_past_days = 100 * sum(list_plus_1_day_low_price_offset_past_days) / float(len(list_plus_1_day_low_price_offset_past_days))
        if len(list_plus_1_day_high_price_offset_past_days) > 0:
            average_plus_1_day_high_price_offset_past_days = 100 * sum(list_plus_1_day_high_price_offset_past_days) / float(len(list_plus_1_day_high_price_offset_past_days))
        if len(list_plus_2_day_low_price_offset_past_days) > 0:
            average_plus_2_day_low_price_offset_past_days = 100 * sum(list_plus_2_day_low_price_offset_past_days) / float(len(list_plus_2_day_low_price_offset_past_days))
        if len(list_plus_2_day_high_price_offset_past_days) > 0:
            average_plus_2_day_high_price_offset_past_days = 100 * sum(list_plus_2_day_high_price_offset_past_days) / float(len(list_plus_2_day_high_price_offset_past_days))

        # write past days offsets
        dict_output['plus_1_day_low_price_offset_past_days'] = '{:.2f}'.format(average_plus_1_day_low_price_offset_past_days)
        dict_output['plus_1_day_high_price_offset_past_days'] = '{:.2f}'.format(average_plus_1_day_high_price_offset_past_days)
        dict_output['plus_2_day_low_price_offset_past_days'] = '{:.2f}'.format(average_plus_2_day_low_price_offset_past_days)
        dict_output['plus_2_day_high_price_offset_past_days'] = '{:.2f}'.format(average_plus_2_day_high_price_offset_past_days)
        
        # write predict values
        this_predict_date_group_string = list_date_group_string[-1]
        list_temp = [x for x in list_predict_daily_data if x['symbol'] == this_symbol and x['date_group'] == this_predict_date_group_string]
        if len(list_temp) == 1:
            this_predict_data_of_this_date_group = list_temp[0]
        else:
            continue
        dict_output['plus_1_day_low_price_predict'] = '{:.2f}'.format(float(this_predict_data_of_this_date_group['plus_1_day_low_price']))
        dict_output['plus_1_day_high_price_predict'] = '{:.2f}'.format(float(this_predict_data_of_this_date_group['plus_1_day_high_price']))
        dict_output['plus_2_day_low_price_predict'] = '{:.2f}'.format(float(this_predict_data_of_this_date_group['plus_2_day_low_price']))
        dict_output['plus_2_day_high_price_predict'] = '{:.2f}'.format(float(this_predict_data_of_this_date_group['plus_2_day_high_price']))
        
        # append to output list
        list_output_datas.append(dict_output)
        
    
    today = datetime.datetime.today().strftime('%Y_%m_%d')
    output_dir = './daily_report'            
    output_file_path = output_dir + '/' + 'daily_report_'+ today +'.csv'            
    list_header = ['symbol', 
                   'plus_1_day_low_price_offset_past_days', 
                   'plus_1_day_high_price_offset_past_days', 
                   'plus_2_day_low_price_offset_past_days', 
                   'plus_2_day_high_price_offset_past_days', 
                   'plus_1_day_low_price_predict', 
                   'plus_1_day_high_price_predict', 
                   'plus_2_day_low_price_predict', 
                   'plus_2_day_high_price_predict'
                  ]
    write_dict_to_csv('w', list_output_datas, list_header, output_file_path )      
   
        
    print('\n\nDone~')

    




process 1101 for 20180815_20180816_20180817
	no predictable data.


process 1102 for 20180815_20180816_20180817
	20180808_20180809_20180810 no real data.
	20180809_20180810_20180813 no real data.
	20180810_20180813_20180814 no real data.
	20180813_20180814_20180815 no real data.
list_plus_2_day_low_price_offset_past_days: 0


process 1103 for 20180815_20180816_20180817
	no predictable data.


process 1104 for 20180815_20180816_20180817
	no predictable data.


process 1108 for 20180815_20180816_20180817
	no predictable data.


process 1109 for 20180815_20180816_20180817
	no predictable data.


process 1110 for 20180815_20180816_20180817
	no predictable data.


process 1201 for 20180815_20180816_20180817
	20180808_20180809_20180810 no real data.
	20180809_20180810_20180813 no real data.
	20180810_20180813_20180814 no real data.
	20180813_20180814_20180815 no real data.
list_plus_2_day_low_price_offset_past_days: 0


process 1203 for 20180815_20180816_20180817
	no predictable data.


pr

	20180813_20180814_20180815 no real data.
list_plus_2_day_low_price_offset_past_days: 0


process 1440 for 20180815_20180816_20180817
	no predictable data.


process 1441 for 20180815_20180816_20180817
	no predictable data.


process 1442 for 20180815_20180816_20180817
	no predictable data.


process 1443 for 20180815_20180816_20180817
	20180808_20180809_20180810 no real data.
	20180809_20180810_20180813 no real data.
	20180810_20180813_20180814 no real data.
	20180813_20180814_20180815 no real data.
list_plus_2_day_low_price_offset_past_days: 0


process 1444 for 20180815_20180816_20180817
	no predictable data.


process 1445 for 20180815_20180816_20180817
	no predictable data.


process 1446 for 20180815_20180816_20180817
	no predictable data.


process 1447 for 20180815_20180816_20180817
	20180808_20180809_20180810 no real data.
	20180809_20180810_20180813 no real data.
	20180810_20180813_20180814 no real data.
	20180813_20180814_20180815 no real data.
list_plus_2_day_low_price_offs

	20180809_20180810_20180813 no real data.
	20180810_20180813_20180814 no real data.
	20180813_20180814_20180815 no real data.
list_plus_2_day_low_price_offset_past_days: 0


process 1591 for 20180815_20180816_20180817
	no predictable data.


process 1592 for 20180815_20180816_20180817
	no predictable data.


process 1593 for 20180815_20180816_20180817
	no predictable data.


process 1595 for 20180815_20180816_20180817
	no predictable data.


process 1597 for 20180815_20180816_20180817
	20180808_20180809_20180810 no real data.
	20180809_20180810_20180813 no real data.
	20180810_20180813_20180814 no real data.
	20180813_20180814_20180815 no real data.
list_plus_2_day_low_price_offset_past_days: 0


process 1598 for 20180815_20180816_20180817
	20180808_20180809_20180810 no real data.
	20180809_20180810_20180813 no real data.
	20180810_20180813_20180814 no real data.
	20180813_20180814_20180815 no real data.
list_plus_2_day_low_price_offset_past_days: 0


process 1599 for 20180815_20180816

	20180813_20180814_20180815 no real data.
list_plus_2_day_low_price_offset_past_days: 0


process 2002 for 20180815_20180816_20180817
	no predictable data.


process 2006 for 20180815_20180816_20180817
	no predictable data.


process 2007 for 20180815_20180816_20180817
	no predictable data.


process 2008 for 20180815_20180816_20180817
	no predictable data.


process 2009 for 20180815_20180816_20180817
	no predictable data.


process 2010 for 20180815_20180816_20180817
	no predictable data.


process 2012 for 20180815_20180816_20180817
	no predictable data.


process 2013 for 20180815_20180816_20180817
	20180808_20180809_20180810 no real data.
	20180809_20180810_20180813 no real data.
	20180810_20180813_20180814 no real data.
	20180813_20180814_20180815 no real data.
list_plus_2_day_low_price_offset_past_days: 0


process 2014 for 20180815_20180816_20180817
	20180808_20180809_20180810 no real data.
	20180809_20180810_20180813 no real data.
	20180810_20180813_20180814 no real data.
	201



process 2340 for 20180815_20180816_20180817
	20180808_20180809_20180810 no real data.
	20180809_20180810_20180813 no real data.
	20180810_20180813_20180814 no real data.
	20180813_20180814_20180815 no real data.
list_plus_2_day_low_price_offset_past_days: 0


process 2342 for 20180815_20180816_20180817
	20180808_20180809_20180810 no real data.
	20180809_20180810_20180813 no real data.
	20180810_20180813_20180814 no real data.
	20180813_20180814_20180815 no real data.
list_plus_2_day_low_price_offset_past_days: 0


process 2344 for 20180815_20180816_20180817
	20180808_20180809_20180810 no real data.
	20180809_20180810_20180813 no real data.
	20180810_20180813_20180814 no real data.
	20180813_20180814_20180815 no real data.
list_plus_2_day_low_price_offset_past_days: 0


process 2345 for 20180815_20180816_20180817
	20180808_20180809_20180810 no real data.
	20180809_20180810_20180813 no real data.
	20180810_20180813_20180814 no real data.
	20180813_20180814_20180815 no real data.
list_p

...

	20180810_20180813_20180814 no real data.
	20180813_20180814_20180815 no real data.
list_plus_2_day_low_price_offset_past_days: 0


process 4930 for 20180815_20180816_20180817
	no predictable data.


process 4933 for 20180815_20180816_20180817
	20180808_20180809_20180810 no real data.
	20180809_20180810_20180813 no real data.
	20180810_20180813_20180814 no real data.
	20180813_20180814_20180815 no real data.
list_plus_2_day_low_price_offset_past_days: 0


process 4934 for 20180815_20180816_20180817
	20180808_20180809_20180810 no real data.
	20180809_20180810_20180813 no real data.
	20180810_20180813_20180814 no real data.
	20180813_20180814_20180815 no real data.
list_plus_2_day_low_price_offset_past_days: 0


process 4935 for 20180815_20180816_20180817
	20180808_20180809_20180810 no real data.
	20180809_20180810_20180813 no real data.
	20180810_20180813_20180814 no real data.
	20180813_20180814_20180815 no real data.
list_plus_2_day_low_price_offset_past_days: 0


process 4938 for 201

	20180810_20180813_20180814 no real data.
	20180813_20180814_20180815 no real data.
list_plus_2_day_low_price_offset_past_days: 0


process 5230 for 20180815_20180816_20180817
	no predictable data.


process 5234 for 20180815_20180816_20180817
	no predictable data.


process 5243 for 20180815_20180816_20180817
	no predictable data.


process 5245 for 20180815_20180816_20180817
	20180808_20180809_20180810 no real data.
	20180809_20180810_20180813 no real data.
	20180810_20180813_20180814 no real data.
	20180813_20180814_20180815 no real data.
list_plus_2_day_low_price_offset_past_days: 0


process 5251 for 20180815_20180816_20180817
	no predictable data.


process 5255 for 20180815_20180816_20180817
	20180808_20180809_20180810 no real data.
	20180809_20180810_20180813 no real data.
	20180810_20180813_20180814 no real data.
	20180813_20180814_20180815 no real data.
list_plus_2_day_low_price_offset_past_days: 0


process 5258 for 20180815_20180816_20180817
	20180808_20180809_20180810 no r

	20180813_20180814_20180815 no real data.
list_plus_2_day_low_price_offset_past_days: 0


process 5392 for 20180815_20180816_20180817
	20180808_20180809_20180810 no real data.
	20180809_20180810_20180813 no real data.
	20180810_20180813_20180814 no real data.
	20180813_20180814_20180815 no real data.
list_plus_2_day_low_price_offset_past_days: 0


process 5398 for 20180815_20180816_20180817
	20180808_20180809_20180810 no real data.
	20180809_20180810_20180813 no real data.
	20180810_20180813_20180814 no real data.
	20180813_20180814_20180815 no real data.
list_plus_2_day_low_price_offset_past_days: 0


process 5403 for 20180815_20180816_20180817
	no predictable data.


process 5410 for 20180815_20180816_20180817
	no predictable data.


process 5425 for 20180815_20180816_20180817
	20180808_20180809_20180810 no real data.
	20180809_20180810_20180813 no real data.
	20180810_20180813_20180814 no real data.
	20180813_20180814_20180815 no real data.
list_plus_2_day_low_price_offset_past_days

	20180809_20180810_20180813 no real data.
	20180810_20180813_20180814 no real data.
	20180813_20180814_20180815 no real data.
list_plus_2_day_low_price_offset_past_days: 0


process 5907 for 20180815_20180816_20180817
	20180808_20180809_20180810 no real data.
	20180809_20180810_20180813 no real data.
	20180810_20180813_20180814 no real data.
	20180813_20180814_20180815 no real data.
list_plus_2_day_low_price_offset_past_days: 0


process 6005 for 20180815_20180816_20180817
	no predictable data.


process 6015 for 20180815_20180816_20180817
	no predictable data.


process 6016 for 20180815_20180816_20180817
	no predictable data.


process 6020 for 20180815_20180816_20180817
	no predictable data.


process 6021 for 20180815_20180816_20180817
	20180808_20180809_20180810 no real data.
	20180809_20180810_20180813 no real data.
	20180810_20180813_20180814 no real data.
	20180813_20180814_20180815 no real data.
list_plus_2_day_low_price_offset_past_days: 0


process 6023 for 20180815_20180816

KeyboardInterrupt: 